In [1]:
import pytest

**Таблица сопряжённости** (матрица неточности) содержит сводные показатели качества работы классификатора.
* **Строки** соответствуют **фактическим** классам тестового набора;
* **Cтолбцы** соответствуют **предсказанным** классом.

Таблица содержит четыре сводных показателя, каждый из которых отражает количество объектов в одной и четырех
категорий:
* **Истинно позитивный** (*True positive*, **TP**) -- объект
класса `1` был верно помечен меткой `1`;
* **Ложно позитивный** (*False positive*, **FP**) -- объект
фактически принадлежит классу `0`, но помечен меткой `1`;
* **Истинно отрицательный** (*True negative*, **TN**) -- классификатор
верно определил, что объект класса `0` принадлежит классу `0`;
* **Ложно отрицательный** (*False negative*, **FN**) -- классификатор
пометил объект меткой `0`, однако на самом деле объект принадлежит классу `1`.


|                    | Предсказано `0` | Предсказано `1` |
|:-------------------|:----------------|:----------------|
| **Фактически** `0` | TN              | FP              |
| **Фактически** `1` | FN              | TP              |


In [2]:
import numpy as np


def get_confusion_matrix(y_true, y_pred):
    """
    :param y_true: реальные классы
    :param y_pred: предсказанные классы
    """
    confusion_matrix = np.array([[0,0], [0,0]]) # столбец - предсказанный класс; строка - реальный
    for value, prediction in zip(y_true, y_pred):
        confusion_matrix[value][prediction] += 1
    return confusion_matrix


def f1_score(y_true, y_pred):
    TN, FP, FN, TP = get_confusion_matrix(y_true, y_pred).ravel()
    return TP / (TP + (FP + FN) / 2)


def accuracy_score(y_true, y_pred):
    TN, FP, FN, TP = get_confusion_matrix(y_true, y_pred).ravel()
    return (TN + TP) / (TN + FP + FN + TP)

In [3]:
import sklearn
import math

def test_computations(y_true, y_pred):
    assert math.isclose(f1_score(y_true, y_pred), sklearn.metrics.f1_score(y_true, y_pred))
    assert math.isclose(accuracy_score(y_true, y_pred), sklearn.metrics.accuracy_score(y_true, y_pred))

In [ ]:
y_true = [0,1,1,0,0,1,1,0,0,0]
y_pred = [0,1,1,0,0,1,0,1,0,1]
test_computations(y_true, y_pred)